In [1]:
import re
import os
import spacy
from datetime import datetime
import xml.etree.ElementTree as ET

## XML Reader

In [2]:
def read_xml(file_name):

    # LOADING XML AND CREATING ROOT
    xml_file = f'{file_name}'
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # EXTRACTING METADATA (TITLE, PUBLICATION_DATE and DOI)
    metadata = []

    for elem in root:
        if elem.tag[29:] == 'teiHeader':
            for sub_elem1 in elem:
                if sub_elem1.tag[29:] == 'fileDesc':
                    publication_info = {}
                    for sub_elem2 in sub_elem1:
                        if sub_elem2.tag[29:] == 'titleStmt':
                            for sub_elem3 in sub_elem2:
                                if sub_elem3.tag[29:] == 'title':
                                    publication_info['Title'] = sub_elem3.text
                        if sub_elem2.tag[29:] == 'publicationStmt':
                            for sub_elem3 in sub_elem2:
                                if sub_elem3.tag[29:] == 'date':
                                    publication_info['Publication Date'] = sub_elem3.attrib['when']
                        if sub_elem2.tag[29:] == 'sourceDesc':
                            for sub_elem3 in sub_elem2:
                                if sub_elem3.tag[29:] == 'biblStruct':
                                    for sub_elem4 in sub_elem3:
                                        if sub_elem4.tag[29:] == 'idno':
                                            publication_info['DOI'] = sub_elem4.text
                    metadata.append(['0', 'Metadata', publication_info])
    
    # EXTRACTING ABSTRACT AND NUMBERING THE PARAGRAPHS
    abstract = []

    for elem in root:
        if elem.tag[29:] == 'teiHeader':
            for sub_elem1 in elem:
                if sub_elem1.tag[29:] == 'profileDesc':
                    for sub_elem2 in sub_elem1:
                        if sub_elem2.tag[29:] == 'abstract':
                            for sub_elem3 in sub_elem2:
                                if sub_elem3.tag[29:] == 'div':
                                    list_of_paragraphs = {}
                                    paragraph_number = 0
                                    for sub_elem4 in sub_elem3:
                                        if sub_elem4.tag[29:] == 'p':
                                            paragraph_number += 1
                                            list_of_paragraphs[paragraph_number] = ET.tostring(sub_elem4, encoding='unicode')
                                    abstract.append(['0', 'Abstract', list_of_paragraphs])
                                 
    # EXTRACTING SECTIONS
    # need to normalize the section number (line 17)
    # compare Rojas and Wolf section number
    # \ issue in RDF, see Koshkava 2014 paper
    list_of_sections = []

    for elem in root:
        if (elem.tag[29:] == 'text'):
            for sub_elem1 in elem:
                if (sub_elem1.tag[29:] == 'body'):
                    for sub_elem2 in sub_elem1:
                        if sub_elem2.tag[29:] == 'div':
                            section_number = ''
                            section_name = ''
                            list_of_paragraphs = []
                            for sub_elem3 in sub_elem2:
                                if sub_elem3.tag[29:] == 'head':
                                    if bool(sub_elem3.attrib):
                                        section_number = str(sub_elem3.attrib)
                                        if section_number[-3] == '.':
                                            section_number = section_number[7:-3]
                                        else:
                                            section_number = section_number[7:-2]
                                    else:
                                        section_number = 'NO_SECTION_NUMBER'
                                    section_name = sub_elem3.text
                                if sub_elem3.tag[29:] == 'p':
                                    list_of_paragraphs.append(ET.tostring(sub_elem3, encoding='unicode'))
                            # commented logic skips NO_SECTION_NUMBER with no paragraphs, i.e., Table 1, Table 2, .....
                            # if section_number == 'NO_SECTION_NUMBER' and not bool(list_of_paragraphs):
                            if section_number == 'NO_SECTION_NUMBER':
                                pass
                            else:
                                list_of_sections.append([section_number, section_name, list_of_paragraphs])
                                
    # NUMBERING THE PARAGRAPHS OF SECTIONS 
    for section in list_of_sections:
        dict_ = {i + 1: section[2][i] for i in range(len(section[2]))}
        section.append(dict_)
        section.remove(section[2])
        
    # MERGING ABSTRACT WITH OTHER DOCUMENT PARTS
    document = abstract + list_of_sections

    # PREPROCESSING TEXT
    starting_p_tag_pattern = r'<ns0:p[^>]+>'
    ending_p_tag_pattern = '</ns0:p>'
    starting_ref_tag_pattern = r'<ns0:ref[^>]+>'
    ending_ref_tag_pattern = '</ns0:ref>'
    ref_pattern = r'<ref>.*?</ref>'          # temporary for removing ref tag

    for record in document:
        for paragraph_number, paragraph_text in record[2].items():
            text = paragraph_text
            text = re.sub(starting_p_tag_pattern, '', text)
            text = re.sub(ending_p_tag_pattern, '', text)
            text = re.sub(starting_ref_tag_pattern, '<ref>', text)
            text = re.sub(ending_ref_tag_pattern, '</ref>', text)
            text = re.sub(ref_pattern, '', text)
            record[2][paragraph_number] = text
    
    # MERGING METADATA WITH OTHER DOCUMENT PARTS
    document = metadata + document
    
    return document


## Necessary Functions for:
***represent_document()*** \
***represent_entity()***

In [3]:
# DEFINING PREFIXES
def define_prefix():

    PREFIX_ONNER = "@prefix onner: <http://purl.org/spatialai/onner/onner-full#> .\n"
    PREFIX_DATA = "@prefix data: <http://purl.org/spatialai/onner/onner-full/data#> .\n"
    PREFIX_RDF = "@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n"
    PREFIX_RDFS = "@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n"
    PREFIX_XSD = "@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n"
    PREFIX_OWL = "@prefix owl: <http://www.w3.org/2002/07/owl#> .\n\n"
    
    return PREFIX_ONNER + PREFIX_DATA + PREFIX_RDF + PREFIX_RDFS + PREFIX_XSD + PREFIX_OWL


# SUBSECTIONS CHECKER FOR A SECTION
def check_nested_section(document, doi, section_number):

    section_ids = []

    for record in document:
        pattern = rf'^{section_number}\.[^.]+$'
        
        if re.search(pattern, record[0]):
            section_id = doi + '_' + str(record[0])
            section_ids.append('data:'+section_id)

    section_ids_joined = ', '.join(section_ids)

    return section_ids_joined


# NLP MODEL LOADER
def ner(paragraph_text):
    
    if not hasattr(ner, 'nlp'):
        try:
            model_name = '/home/umayer/Work/_dev/output_sci_md_vector/model-best'
            ner.nlp = spacy.load(f'{model_name}')
            print('✔️ Model loaded successfully.')
        except OSError:
            print('❌ ERROR: Model Not Found!')

    doc = ner.nlp(paragraph_text) 
    
    return doc


## RDF - Document Representation

In [4]:
# RDF WRITER FOR DOCUMENT
def represent_document(document):

    # GETTING DOI
    try:
        doi = document[0][2]['DOI']
        document_id = doi.replace('/', '_')
    except NameError:
        print('❌ ERROR: Document object is not defined!')

    # PRINTING PREFIXES
    rdf_triple = define_prefix()

    # WRITING RDF
    paragraphs_in_document = []
    
    for record in document:
        # metadata
        if record[1] == 'Metadata':
            abstract_id = 'data:' + document_id + '_A'
            section_ids = []

            for section_number in document:
                if section_number[0] != '0' and '.' not in section_number[0]:
                    section_id = document_id + '_' + section_number[0]
                    section_ids.append('data:'+section_id)

            directly_contained_sections = ', '.join(section_ids)

            rdf_triple += f"data:Publication_{document_id} rdf:type onner:ScholarlyPublication ;\n"
            rdf_triple += f"onner:publicationTitle '{record[2]['Title']}'^^xsd:string ;\n"
            rdf_triple += f"onner:publicationDate '{record[2]['Publication Date']}'^^xsd:date ;\n"
            rdf_triple += f"onner:doi '{doi}'^^xsd:string ;\n"
            rdf_triple += f"onner:directlyContainsDocumentPart {abstract_id}, {directly_contained_sections} .\n\n"

        # abstract and it's document parts
        elif record[1] == 'Abstract':
            next_index = document.index(record) + 1
            next_section = document[next_index][0]
            paragraph_ids = []

            for paragraph_number, _ in record[2].items():
                paragraph_id = document_id + '_A' + '-' + str(paragraph_number)
                paragraph_ids.append('data:'+paragraph_id)

            paragraph_ids_joined = ', '.join(paragraph_ids)

            rdf_triple += f"data:{document_id}_A rdf:type onner:Abstract ;\n"
            rdf_triple += f"onner:nextDocumentPart {paragraph_ids[0]} ;\n"    # NEXT DOC PART AFTER ABSTRACT
            rdf_triple += f"onner:directlyContainsDocumentPart {paragraph_ids_joined} .\n\n"

            for paragraph_number, paragraph_text in record[2].items():               
                # replacing ' with \' in text
                if "'" in paragraph_text:
                    paragraph_text = paragraph_text.replace("'", r"\'")

                # storing paragraph id and paragraph text
                paragraphs_in_document.append([f'{document_id}_A-{paragraph_number}', paragraph_text])
                
                rdf_triple += f"data:{document_id}_A-{paragraph_number} rdf:type onner:Paragraph ;\n"
                rdf_triple += f"onner:positionInParentDocumentPart '{paragraph_number}'^^xsd:nonNegativeInteger ;\n"

                if paragraph_number == len(paragraph_ids):
                    rdf_triple += f"onner:nextDocumentPart data:{document_id}_{next_section} ;\n"
                else:
                    rdf_triple += f"onner:nextDocumentPart data:{document_id}_A-{paragraph_number+1} ;\n"

                rdf_triple += f"onner:paragraphText '{paragraph_text}'^^xsd:string .\n\n"

        # section and it's document parts
        else:
            section_number = record[0]
            section_name = record[1]
            next_index = document.index(record) + 1
            paragraph_ids = []

            if next_index == len(document):
                next_section = 'EndOfDocument'
            else:
                next_section = document[next_index][0]

            # if no paragraphs exist between a section and its immediate subsection
            # else paragraphs exist between a section and its immediate subsection
            if not bool(record[2]):
                directly_contained_sections = check_nested_section(document, document_id, section_number)

                rdf_triple += f"data:{document_id}_{section_number} rdf:type onner:Section ;\n"
                rdf_triple += f"onner:sectionTitle '{section_name}'^^xsd:string ;\n"
                rdf_triple += f"onner:sectionNumber '{section_number}'^^xsd:string ;\n"
                rdf_triple += f"onner:nextDocumentPart data:{document_id}_{next_section} ;\n"
                rdf_triple += f"onner:directlyContainsDocumentPart {directly_contained_sections} .\n\n"
            else:
                directly_contained_sections = check_nested_section(document, document_id, section_number)

                for paragraph_number, _ in record[2].items():
                    paragraph_id = document_id + '_' + str(section_number) + '-' + str(paragraph_number)
                    paragraph_ids.append('data:'+paragraph_id)

                paragraph_ids_joined = ', '.join(paragraph_ids)

                rdf_triple += f"data:{document_id}_{section_number} rdf:type onner:Section ;\n"
                rdf_triple += f"onner:sectionTitle '{section_name}'^^xsd:string ;\n"
                rdf_triple += f"onner:sectionNumber '{section_number}'^^xsd:string ;\n"
                rdf_triple += f"onner:nextDocumentPart {paragraph_ids[0]} ;\n"    # NEXT DOC PART AFTER SECTION

                if bool(directly_contained_sections):
                    rdf_triple += f"onner:directlyContainsDocumentPart {paragraph_ids_joined}, {directly_contained_sections} .\n\n"
                else:
                    rdf_triple += f"onner:directlyContainsDocumentPart {paragraph_ids_joined} .\n\n"

                for paragraph_number, paragraph_text in record[2].items():
                    # replacing ' with \' in text
                    if "'" in paragraph_text:
                        paragraph_text = paragraph_text.replace("'", r"\'")
                      
                    # storing paragraph id and paragraph text
                    paragraphs_in_document.append([f'{document_id}_{section_number}-{paragraph_number}', paragraph_text])
 
                    rdf_triple += f"data:{document_id}_{section_number}-{paragraph_number} rdf:type onner:Paragraph ;\n"
                    rdf_triple += f"onner:positionInParentDocumentPart '{paragraph_number}'^^xsd:nonNegativeInteger ;\n"

                    if paragraph_number == len(paragraph_ids):
                        rdf_triple += f"onner:nextDocumentPart data:{document_id}_{next_section} ;\n"
                    else:
                        rdf_triple += f"onner:nextDocumentPart data:{document_id}_{section_number}-{paragraph_number+1} ;\n"

                    rdf_triple += f"onner:paragraphText '{paragraph_text}'^^xsd:string .\n\n"

    rdf_triple += f"data:{document_id}_EndOfDocument rdf:type onner:EndOfDocument .\n"

    return document_id, paragraphs_in_document, rdf_triple


## RDF - Named Entity Representation

In [5]:
# RDF WRITER FOR NAMED ENTITIES
def represent_entity(paragraphs_in_document):
    
    # GETTING NAMED ENTITIES
    entities_in_document = []
    
    for paragraph in paragraphs_in_document:
        paragraph_id = paragraph[0]
        paragraph_text = paragraph[1]
        
        doc = ner(paragraph_text)
        
        entities_in_paragraph = []
        entity_number = 1
        
        for ent in doc.ents:
            date_time = str(datetime.now())[:-7]    # RECHECK THE APPROPRIATE PLACEMENT
            entity_id = paragraph_id + '-' + str(entity_number)
            entity = ent.text
            label = ent.label_
            offset = ent.start_char
            length = ent.end_char - ent.start_char
            entity_info = [entity_id, entity, label, offset, length, date_time]
            entities_in_paragraph.append(entity_info)
            entity_number += 1

        entities_in_document.append([paragraph_id, entities_in_paragraph])
    
    
    # WRITING RDF
    rdf_triple = define_prefix()
    labels_in_doc = []
    labeling_schema = ['CHEMICAL', 
                       'MATERIAL', 
                       'STRUCTURE',
                       'PROPERTY',
                       'APPLICATION',
                       'PROCESS',
                       'EQUIPMENT',
                       'MEASUREMENT',
                       'ABBREVIATION']
    
    for i in entities_in_document:
        paragraph_id = i[0]
        labeled_term_info_list = i[1]

        if bool(labeled_term_info_list):    # if no term found in paragraph
            entity_ids = ['data:'+i[0] for i in labeled_term_info_list]
            labeled_term_ids_joined = ', '.join(entity_ids) 
            
            rdf_triple += f"data:{paragraph_id} onner:directlyContainsLabeledTerm {labeled_term_ids_joined} .\n\n"
            
            for info in labeled_term_info_list:
                rdf_triple += f"data:{info[0]} rdf:type onner:LabeledTerm ;\n"    # deal with atomic and compound terms
                rdf_triple += f"onner:labeledTermText '{info[1]}'^^xsd:string ;\n"
                rdf_triple += f"onner:offset '{info[3]}'^^xsd:nonNegativeInteger ;\n"
                rdf_triple += f"onner:length '{info[4]}'^^xsd:nonNegativeInteger ;\n"
                rdf_triple += f"onner:labeledTermDirectlyContainedBy data:{paragraph_id} ;\n"
                rdf_triple += f"onner:hasLabeledTermStatus data:Candidate_{info[0]} .\n\n"

                rdf_triple += f"data:Candidate_{info[0]} rdf:type onner:CandidateStatus ;\n"
                rdf_triple += f"onner:statusAssignmentDate '{info[5]}'^^xsd:dateTime ;\n"
                rdf_triple += f"onner:statusAssignedBy data:Cellulosic_NER_Model ;\n"

                try:
                    if not bool(labeling_schema):
                        raise Exception('Labeling schema is empty!')
                    label_number_in_schema = labeling_schema.index(info[2]) + 1
                except Exception as e:
                    print(f'❌ ERROR: {e}')
                else:
                    rdf_triple += f"onner:hasLabeledTermLabel data:Label_{label_number_in_schema} .\n\n"

                # adding lebels and their positions in the schema
                if [label_number_in_schema, info[2]] not in labels_in_doc:
                    labels_in_doc.append([label_number_in_schema, info[2]])
        else:
            rdf_triple += f"data:{paragraph_id} onner:directlyContainsLabeledTerm data:NoLabeledTerm .\n\n"

    try:
        if not bool(labels_in_doc):
            raise Exception('List of labels found in document is empty!\n')     
        else:
            for label in labels_in_doc:
                rdf_triple += f"data:Label_{label[0]} rdf:type onner:Label ;\n"
                rdf_triple += f"onner:fromLabelingSchema data:Labeling_Schema ;\n"
                rdf_triple += f"onner:labelText '{label[1]}'^^xsd:string .\n\n"
    except Exception as e:
        print(f'❌ ERROR: {e}')

    rdf_triple += f"data:Labeling_Schema rdf:type onner:LabelingSchema ;\n"
    rdf_triple += f"onner:schemaName 'CelloGraph'^^xsd:string .\n\n"

    rdf_triple += f"data:Cellulosic_NER_Model rdf:type onner:NER_System ;\n"    # if/else required to identify system and human
    rdf_triple += f"onner:systemVersion '1.0'^^xsd:string .\n\n"

    return rdf_triple


## Main Block

In [6]:
document = read_xml('/home/umayer/_dev/OnNER/evaluation/example/input/Publication_10.1021_acsami.6b14650.xml')
document_id, paragraphs_in_document, rdf_document = represent_document(document)
rdf_entities = represent_entity(paragraphs_in_document)

rdf_outputs = {
    f'/home/umayer/_dev/Publication_{document_id}_document.ttl': rdf_document,
    f'/home/umayer/_dev/Publication_{document_id}_entities.ttl': rdf_entities
}

for turtle_file, rdf_content in rdf_outputs.items():
    with open(turtle_file, 'w') as file:
        file.write(rdf_content)
    print(f'✔️ Turtle saved to {turtle_file}')
    

/home/umayer/Work/_venv/CelloGraph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/umayer/Work/_venv/CelloGraph/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/umayer/Work/_venv/CelloGraph/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


✔️ Model loaded successfully.
✔️ Turtle saved to /home/umayer/_dev/Publication_10.1021_acsami.6b14650_document.ttl
✔️ Turtle saved to /home/umayer/_dev/Publication_10.1021_acsami.6b14650_entities.ttl


## Sample Output
### RDF for Document

In [7]:
print(rdf_document)


@prefix onner: <http://purl.org/spatialai/onner/onner-full#> .
@prefix data: <http://purl.org/spatialai/onner/onner-full/data#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

data:Publication_10.1021_acsami.6b14650 rdf:type onner:ScholarlyPublication ;
onner:publicationTitle 'Macrofibers with High Mechanical Performance Based on Aligned Bacterial Cellulose Nanofibers'^^xsd:string ;
onner:publicationDate '2017-01-03'^^xsd:date ;
onner:doi '10.1021/acsami.6b14650'^^xsd:string ;
onner:directlyContainsDocumentPart data:10.1021_acsami.6b14650_A, data:10.1021_acsami.6b14650_1, data:10.1021_acsami.6b14650_2, data:10.1021_acsami.6b14650_3, data:10.1021_acsami.6b14650_4 .

data:10.1021_acsami.6b14650_A rdf:type onner:Abstract ;
onner:nextDocumentPart data:10.1021_acsami.6b14650_A-1 ;
onner:directlyContainsDocumentPart data:10

### RDF for Named Entities

In [8]:
print(rdf_entities)


@prefix onner: <http://purl.org/spatialai/onner/onner-full#> .
@prefix data: <http://purl.org/spatialai/onner/onner-full/data#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

data:10.1021_acsami.6b14650_A-1 onner:directlyContainsLabeledTerm data:10.1021_acsami.6b14650_A-1-1, data:10.1021_acsami.6b14650_A-1-2, data:10.1021_acsami.6b14650_A-1-3, data:10.1021_acsami.6b14650_A-1-4, data:10.1021_acsami.6b14650_A-1-5, data:10.1021_acsami.6b14650_A-1-6, data:10.1021_acsami.6b14650_A-1-7, data:10.1021_acsami.6b14650_A-1-8, data:10.1021_acsami.6b14650_A-1-9, data:10.1021_acsami.6b14650_A-1-10, data:10.1021_acsami.6b14650_A-1-11, data:10.1021_acsami.6b14650_A-1-12, data:10.1021_acsami.6b14650_A-1-13, data:10.1021_acsami.6b14650_A-1-14, data:10.1021_acsami.6b14650_A-1-15, data:10.1021_acsami.6b14650_A-1-16, data:10.1021_acsami.